This is baseline !!!!!!
---------------------------

Old data set
-------------

In [1]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
from code.dataExtract import data
import code.dataExtract as de

data0='../data_0'
data1='../data_1'
labels=label.from_cvs(data0)
data=copy.deepcopy(labels)
data.empty()

fcomp = fuzzCompany2.from_file('fc_data.pkl')

The PythonMagick package is missing, get_text_adv function can not be used 
['fileId', 'NOMINAL.CURR', 'MIN.TRAD.AMT', 'MULT.TRAD.AMT', 'ZERO.COUPN.FLAG', 'SEC.SUB.ID', 'FUNG.FL', 'OpCurrency', 'issuerName', 'issuerCity', 'issuerCountry', 'guarantorName', 'guarantorCity', 'guarantorCountry', 'City.Id', 'City.Name']


In [5]:
print_info=False

def get_issuer(txt):
    
    for s in tr.split_text(txt):   # split into sentences.  
        
        s = s.lower()
        if print_info:
            print repr(s)
                
        if re.search(r'.*issuer[\s\n]*:.*', s.lower()) or re.match(r'issuer\s{4,}.*', s.lower()):            
            tmp = fcomp.closerMatch(s.replace('issuer',' '))
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, s.replace('issuer','')
                if value > 0.1:
                    return name
            
        test=re.findall(r'issued by.*', s.lower())
        if len(test) > 0:
            test=test[0].split('\n')[0].replace('issued by','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
        
        test=re.findall(r'.*issue of', s.lower())
        if len(test) > 0:
            test=test[0].replace('issue of','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value,name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
            
        test=re.findall(r'.*issued', s.lower())
        if len(test) > 0:
            test=test[0].replace('issue','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
    
    return None

In [6]:
folder='../data_0/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../data_0/txt')

for isin in data: # for each isin in the data object created before
#for isin in ['XS1190873098']:
    f=data[isin][0] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    print isin, issuer
    if issuer is not None:
        data[isin][8]=issuer

XS1240015799 CREDIT SUISSE INTERNATIONAL
XS1293137904 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1313796846 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1313802677 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1313800895 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1267246327 GOLDMAN SACHS INTERNATIONAL
XS1275964507 DEUTSCHE BANK AG
XS1329979261 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1312160507 ROYAL BANK OF CANADA
XS1253932765 CREDIT SUISSE INTERNATIONAL
XS1267246244 GOLDMAN SACHS INTERNATIONAL
XS1293817877 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1051370440 DEUTSCHE BANK AG
XS1267221841 GOLDMAN SACHS INTERNATIONAL
XS1266266813 BARCLAYS BANK PLC
XS1313804459 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1332626016 ROYAL BANK OF CANADA
XS1338871996 SMBC NIKKO SECURITIES INC.
XS1051380837 None
XS1313800119 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1293807720 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1326568430 ROYAL BANK OF CANADA
XS1222904416 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1339630227 R

In [7]:
data.evaluate([8],labels,printerr=True, match_ok=100)

XS1051380837 ,  None ,  DEUTSCHE BANK AG
XS1324899985 ,  None ,  HSBC BANK PLC
XS1336933194 ,  None ,  HSBC BANK PLC
XS1183228789 ,  NATIONAL AUSTRALIA BANK LTD ,  CREDIT SUISSE INTERNATIONAL
XS1336567315 ,  None ,  HSBC BANK PLC
XS1332081352 ,  None ,  HSBC BANK PLC
XS1051372495 ,  None ,  DEUTSCHE BANK AG
XS0461332347 ,  None ,  DEUTSCHE BANK AG
XS1330994689 ,  None ,  ROYAL BANK OF CANADA
XS1051364633 ,  None ,  DEUTSCHE BANK AG
XS1254327510 ,  STANDARD CHARTERED PLC ,  CREDIT SUISSE INTERNATIONAL
XS1340707659 ,  None ,  HSBC BANK PLC
XS1340128625 ,  None ,  HSBC BANK PLC
XS1349641206 ,  None ,  HSBC BANK PLC
XS1051365010 ,  None ,  DEUTSCHE BANK AG
XS1051340963 ,  None ,  DEUTSCHE BANK AG
XS1051371414 ,  None ,  DEUTSCHE BANK AG
XS1221134189 ,  None ,  DEUTSCHE BANK AG
XS1313797141 ,  None ,  BNP PARIBAS ARBITRAGE ISSUANCE
XS1324210563 ,  None ,  HSBC BANK PLC
XS1354473230 ,  None ,  HSBC BANK PLC
XS1330444420 ,  None ,  HSBC BANK PLC
XS1323908944 ,  None ,  HSBC BANK PLC
DE000UZ16

New data set
-------------

In [25]:
# Make the model for company.

#issuerset=set()
#for item in d.issuer:
#    issuerset.update(d.issuer[item])
from code.Matching import fuzzCompany2
import ordered_set

# get all the companies for the referance data
with open('company.csv','r') as f:
    c = f.readlines()
companies =  ordered_set.OrderedSet()

for item in c[1:]:
    companies.add(item.strip('\n", ')) #.split(',')[0])

syn=[('J.P. MORGAN','JPMORGAN'), ('JPMORGAN','J.P. MORGAN'),('J.P.MORGAN','J.P. MORGAN'), ('aktiengesellschaft','ag'),
    ('Limited liability company','llc'),('Public limited company','plc'),('incorporation','inc'),
    ('Sociedad por Acciones','sa'),('Public Limited','ltd'),('int','international'), ('london','bishopsgate (110)'),
    ('paris','fort dup 11046'),('london','great winche'),('london','golden lane'),('new world development','nwd'),
    ('england ','bishopsgate (110)'),
    ('london','bishopsgate (110)'),
    ('acting through its london branch','great winche'),
    ('acting through its london branch','golden lane'),
    ('incorporated with limited liability in england and wales','bishopsgate (110)')
    ]

fcomp = fuzzCompany2(companies,syn)

In [26]:
import cPickle as pickle
with open('fc_data2.pkl', 'wb') as output:
    pickle.dump(fcomp, output)

In [16]:
'NEW WORLD DEVELOPMENT'.lower()

'new world development'

In [27]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
from code.dataExtract import data
import code.dataExtract as de

d = data(folder='../Newdata/train/')

fcomp = fuzzCompany2.from_file('fc_data2.pkl')

In [31]:
print_info=False
#print_info=True

def get_issuer(txt):
    
    for s in tr.split_text(txt):   # split into sentences.  
        
        s = s.lower()
                
        if re.search(r'.*issuer[\s\n]*:.*', s.lower()) or re.match(r'issuer\s{4,}.*', s.lower()):            
            tmp = fcomp.closerMatch(s.replace('issuer',' '))
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, s.replace('issuer','')
                if value > 0.1:
                    return name
            
        test=re.findall(r'issued by.*', s.lower())
        if len(test) > 0:
            test=test[0].split('\n')[0].replace('issued by','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
        
        test=re.findall(r'.*issue of', s.lower())
        if len(test) > 0:
            test=test[0].replace('issue of','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value,name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
            
        test=re.findall(r'.*issued', s.lower())
        if len(test) > 0:
            test=test[0].replace('issue','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
    
    return None

In [36]:
folder='../Newdata/train/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../Newdata/train/txt')

issuerDic={}

x=0
for isin in d.docid: # for each isin in the data object created before
#for isin in ['XS1069359716']:
    if x > 6000:
        break
    x+=1
    
    if x%100 == 0:
        print x
    
    f=d.docid[isin] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    #print isin, issuer
    if issuer is not None:
        issuerDic[isin]=issuer
    else:
        issuerDic[isin]=''

KeyboardInterrupt: 

In [10]:
print fcomp.values[1306]
print fcomp.l[1306]

UBS AG
2.0


In [9]:
print fcomp.values[1307]
print fcomp.l[1307]

UBS AG, GOLDEN LANE
4.0


In [37]:
len(issuerDic)

4596

In [38]:
issuerAns = {}
x=0
for item in d.issuer:
    if item in issuerDic:    
        issuerAns[item] = d.issuer[item][0]
    
de.test(issuerDic, issuerAns)


total number of set compared: 4596.0
total number of missing set: 0.0
error %: 23.2158398607


In [39]:
for item in issuerDic:
    if issuerDic[item] != issuerAns[item] and issuerDic[item]!='':
        print item, d.docid[item],' , ',issuerDic[item],' , ',issuerAns[item]

US05579HAB87 OrderedSet(['0900045c83d95b93'])  ,  RESERVE BANK OF NEW ZEALAND  ,  BNZ INTERNATIONAL FUNDING LIMITED
XS0875804667 OrderedSet(['0900045c812d9f81'])  ,  AB VOLVOFINANS  ,  SWEDISH EXPORT CREDIT CORP
CH0281105335 OrderedSet(['0900045c831edcaf'])  ,  EXANE DERIVATIVES  ,  EXANE FINANCE
CH0295931650 OrderedSet(['0900045c839794f3'])  ,  EXANE DERIVATIVES  ,  EXANE FINANCE
XS1267602214 OrderedSet(['0900045c834e60c4', '0900045c834e60cd'])  ,  CHINA OILFIELD SERVICES LIMITED  ,  COSL SINGAPORE CAPITAL LTD.
FR0013080074 OrderedSet(['0900045c83abbd83'])  ,  EXANE DERIVATIVES  ,  EXANE FINANCE
XS0941235854 OrderedSet(['0900045c817f822c'])  ,  BAXALTA INCORPORATED  ,  CITIGROUP INC
CH0295931270 OrderedSet(['0900045c838be64a'])  ,  EXANE DERIVATIVES  ,  EXANE FINANCE
XS1337093121 OrderedSet(['0900045c83c31527'])  ,  KINGDOM OF THE NETHERLANDS  ,  TECHNOLOGY ENHANCED OIL PLC
XS1183228789 OrderedSet(['0900045c82d3b33c'])  ,  NATIONAL AUSTRALIA BANK LTD  ,  CREDIT SUISSE INTERNATIONAL
XS

validation data set
---------------------

In [1]:
from code.Matching import fuzzCompany2
import os
from code import Text_reader as tr
from code.Labels import label
from code.Text_reader import htmlDoc
import copy 
import re
from code.dataExtract import data
import code.dataExtract as de

d = data(folder='../Newdata/int_test/')

fcomp = fuzzCompany2.from_file('fc_data2.pkl')

The PythonMagick package is missing, get_text_adv function can not be used 


In [2]:
print_info=False

def get_issuer(txt):
    
    for s in tr.split_text(txt):   # split into sentences.  
        
        s = s.lower()
                
        if re.search(r'.*issuer[\s\n]*:.*', s.lower()) or re.match(r'issuer\s{4,}.*', s.lower()):            
            tmp = fcomp.closerMatch(s.replace('issuer',' '))
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, s.replace('issuer','')
                if value > 0.1:
                    return name
            
        test=re.findall(r'issued by.*', s.lower())
        if len(test) > 0:
            test=test[0].split('\n')[0].replace('issued by','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
        
        test=re.findall(r'.*issue of', s.lower())
        if len(test) > 0:
            test=test[0].replace('issue of','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value,name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
            
        test=re.findall(r'.*issued', s.lower())
        if len(test) > 0:
            test=test[0].replace('issue','')
            tmp = fcomp.closerMatch(test)
            if tmp is not None:
                value, name = tmp
                if print_info:
                    print name, value, test
                if value > 0.1:
                    return name
    
    return None

In [3]:
folder='../Newdata/int_test/html'
files=tr.get_files(folder)
filesTxT=tr.get_files('../Newdata/int_test/txt')

issuerDic={}

x=0
for isin in d.docid: # for each isin in the data object created before
    
    f=d.docid[isin] # get the file_ids
    issuer=None
    for item in f: # for each of these files
        # get the txt.
        txt=tr.clean_text(tr.get_text(filesTxT[item],raw=True))
        issuer=get_issuer(txt)
        if issuer is None:
            txt=tr.clean_text(tr.get_text(files[item]))
            issuer=get_issuer(txt)
            
        if issuer is not None:
            break
    
    print isin, issuer
    if issuer is not None:
        issuerDic[isin]=issuer
    else:
        issuerDic[isin]=''

FR0013149382 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1390266838 SMBC NIKKO SECURITIES INC.
XS1378402199 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
CH0285315344 EXANE DERIVATIVES
XS1138845844 MORGAN STANLEY B.V.
JP1023601G11 CITIGROUP CAPITAL
CH0254071894 None
XS1387752329 SMBC NIKKO SECURITIES INC.
XS1313827740 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1373121745 SMBC NIKKO SECURITIES INC.
XS1328130197 CHINA CONSTRUCTION BANK CORPORATION
XS1328604837 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1369058059 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS1293806839 THE HONGKONG AND SHANGHAI BANKING CORPORATION LIMITED
XS0532879300 BARCLAYS BANK PLC
XS1315761459 SMBC NIKKO SECURITIES INC.
XS1281070455 HSBC BANK PLC
XS1389980613 SMBC NIKKO SECURITIES INC.
XS1319973456 CREDIT SUISSE INTERNATIONAL
XS1314926939 SG ISSUER
XS1317103569 UBS AG
XS1338495218 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS1338526970 BNP PARIBAS ARBITRAGE ISSUANCE BV
XS0934761585 BARCLAYS BANK PLC
XS1384275

In [4]:
de.to_csv('issuer.csv',issuerDic)